### Copyright (C) Microsoft Corporation.    
  
# Deploy regular ML R model in ACI and AKS
  

* Use the user provided R model and R scoring script embedded in the containerized Python operationalization (o16n) script to deloy R model at scale using [Azure Kubernetes Service](https://docs.microsoft.com/en-us/azure/aks/) 

In [ ]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
aci_service_name = 'r-svm-aci-service-01'
aks_service_name ='ro16n-aks-srvc01'
version_of_image_to_deploy = 1
version_of_model_to_deploy = 1

In [ ]:
import azureml.core
import platform
import sys, os
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.image import Image, ContainerImage

def ls_l(a_dir):
    return ([f for f in os.listdir(a_dir) if os.path.isfile(os.path.join(a_dir, f))]) 

In [ ]:
# Check core SDK version number, os info and current wd
print("SDK version:", azureml.core.VERSION)
platform.platform()

In [ ]:
# import utility functions
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)

add_path_to_sys_path(os.path.join(os.getcwd(), os.path.join(*(['.', 'src']))))

import o16n_regular_ML_R_models_utils
prj_consts = o16n_regular_ML_R_models_utils.R_models_operationalization_consts()

#### Define some variables


In [ ]:
# Define project params
prj_consts = o16n_regular_ML_R_models_utils.R_models_operationalization_consts()

experiment_dir = os.path.join(*(prj_consts.AML_EXPERIMENT_DIR))
workspace_config_dir = os.path.join(*(prj_consts.AML_WORKSPACE_CONFIG_DIR))
R_artifacts_dir = os.path.join(os.getcwd(), os.path.join(*(prj_consts.R_MODEL_DIR)))

In [ ]:
# check if we have the right elements for o16n
os.path.isfile( os.path.join(os.getcwd(), os.path.join(experiment_dir, prj_consts.R_MODEL_CONDA_DEPENDENCIES_FILE_NAME)))
os.path.isfile( os.path.join(os.getcwd(), os.path.join(experiment_dir, prj_consts.SCORE_SCRIPT_FILE_NAME)))
os.path.isfile(os.path.join(R_artifacts_dir, prj_consts.R_MODEL_FILE_NAME))

## Initialize Workspace

Initialize a workspace object configuration persisted in previous notebook.

In [ ]:
ws = Workspace.from_config(
    path=os.path.join(os.getcwd(), 
                      os.path.join(*([workspace_config_dir, '.azureml', prj_consts.AML_WORKSPACE_CONFIG_FILE_NAME]))))

# print(ws.name, ws.resource_group, ws.location, ws.subscription_id[0], sep = '\n')

#### We can register a model, and choose one of the registered ones for deployment. This step can be skipped since there should already be a model registered from the previous notebook.

In [ ]:
model_tags = {'language': 'R', 'type': 'TC_kSVM'}
if not Model.list(ws, tags=model_tags):
    model = Model.register(model_path = os.path.join(R_artifacts_dir, prj_consts.R_MODEL_FILE_NAME),
                           model_name = prj_consts.R_MODEL_AML_NAME,
                           tags = model_tags,
                           description = 'my R model',
                           workspace = ws)
    
    print(model.name, model.description, model.version, model.tags, sep = '\t')

You can explore the registered models within your workspace and query by tag. Models are versioned. If you call the register_model command many times with same model name, you will get multiple versions of the model with increasing version numbers.   
For demo purposes, we choose v1 as the model used for deployment.

In [ ]:
best_r_model = None

for m in Model.list(ws, tags={'type': 'TC_kSVM'}):
    print("Name:", m.name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)
    if ((m.name==prj_consts.R_MODEL_AML_NAME) and (m.version==version_of_model_to_deploy) and (m.description=='my R model')):
        best_r_model = m

print(best_r_model.name, best_r_model.description, best_r_model.version, sep = '\t')


#### Print content of operationalization image directory:
 - scoring script (o16n pyth0n script that embeds the user provided R scoring script) 
 - R and python package dependencies decribed in conda environment .yml file 
 - R model file is not necessary in this case, as we will first register it via Azure AML Model Management and thus access it in the cloud

In [ ]:
ls_l(os.path.join(os.getcwd(), os.path.join(experiment_dir)))

#### Print content of conda_dependencies yml file

In [ ]:
! cat {os.path.join(os.getcwd(), os.path.join(*[experiment_dir, prj_consts.R_MODEL_CONDA_DEPENDENCIES_FILE_NAME]))}

## Create o16n image, using registered model

In [ ]:
image_tags = {'area': "R models o16n", 'type': "regular ML"}
if not Image.list(workspace = ws,tags = image_tags):
    crt_dir = os.getcwd()
    os.chdir(os.path.join(os.getcwd(), os.path.join(*[experiment_dir])))


    image_config = ContainerImage.image_configuration(runtime= "python",
                                     execution_script=prj_consts.SCORE_SCRIPT_FILE_NAME,
                                     conda_file=prj_consts.R_MODEL_CONDA_DEPENDENCIES_FILE_NAME,
                                     tags = image_tags,
                                     description = "Image with kSVM R model o16n-ed via rpy2")

    image = Image.create(name = prj_consts.o16n_DOCKER_IMAGE_NAME,
                         # this is the model object 
                         models = [best_r_model],
                         image_config = image_config, 
                         workspace = ws)

    image.wait_for_creation(show_output = True)
    os.chdir(crt_dir)

List images built so far

In [ ]:
image_to_deploy= None
for i in Image.list(workspace = ws,tags = image_tags):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, 
                                                                i.version, 
                                                                i.creation_state, 
                                                                i.image_location, 
                                                                i.image_build_log_uri))
    if ((i.name==prj_consts.o16n_DOCKER_IMAGE_NAME) and (i.version==version_of_image_to_deploy)):
        image_to_deploy = i

print('image_to_deploy:')
print(image_to_deploy.name, image_to_deploy.version, image_to_deploy.image_location, sep = '\t')

## Deploy image as web service on Azure Container Instance

Note that the service creation can take few minutes.

In [ ]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "R models o16n"}, 
                                               description = 'demo R SVM model in AML ACI')

#### List all web services in the workspace

In [ ]:
from azureml.core.webservice import Webservice 

print('name, state, created_time, compute_type, description, scoring_uri, scoring_uri, image_id, image')
for crt_webservice in Webservice.list(workspace = ws):
    print('{}, {}, {}, {}, {}, {}, {}, {}'.format(crt_webservice.name,
                                                  crt_webservice.state,
                                                  crt_webservice.created_time,
                                                  crt_webservice.compute_type,
                                                  crt_webservice.description,
                                                  crt_webservice.scoring_uri,
                                                  crt_webservice.image_id,
                                                  crt_webservice.image.name))


In [ ]:
from azureml.exceptions import WebserviceException

try:
    aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image_to_deploy,
                                           name = aci_service_name,
                                           workspace = ws)
    aci_service.wait_for_deployment(True)
    print(aci_service.state)
except WebserviceException:
    print('There is already a service with name {} found in workspace {}. Will use it, and not create another one!'\
          .format(aci_service_name, ws.name))
    aci_service = Webservice(workspace = ws, name = aci_service_name)

### Test web service
Call the web service with some dummy input data to get a prediction.

In [ ]:
import numpy as np
import json
import pandas as pd

In [ ]:
n_samples = 1000

raw_data = 2 * np.random.random_sample((n_samples, 2)) - 1
if n_samples<10:
    raw_data

aml_jsoned_data =  json.dumps({'data': json.dumps(raw_data.tolist())})
response = aci_service.run(input_data = aml_jsoned_data)

if n_samples<10:
    print( pd.DataFrame.from_records(json.loads(json.loads(response)['python_scores'])['r_scores']) )

print( pd.DataFrame.from_records(json.loads(json.loads(response)['python_scores'])['r_times']) )
for k, v in json.loads(json.loads(response)['python_times']).items():
    print(v, k)

In [ ]:
pd.DataFrame.from_records(json.loads(json.loads(response)['python_scores'])['r_times'])
pd.DataFrame.from_records(json.loads(json.loads(response)['python_scores'])['r_times']).iloc[0,1]

json.loads(json.loads(response)['python_times'])
json.loads(json.loads(response)['python_times'])['all_p_time']

def rpy_times_report(r_times_dataframe, python_times_dict):
    python_time_number, python_time_unit = python_times_dict['all_p_time'].split()
    r_time_number = r_times_dataframe.iloc[0,1]

    for crt_key, crt_value in \
    {'rpy overhead summary':'',
     'r_processing time':'{} ms'.format(round(float(r_time_number)), 2),
     'python_processing time':'{} ms'.format(round(float(python_time_number)), 2),
     'rpy overhead':'{} %'.format(round(((float(python_time_number)-float(r_time_number))/float(r_time_number))*100, 2))}.items():
        print(crt_key, '\t',crt_value)  

rpy_times_report(pd.DataFrame.from_records(json.loads(json.loads(response)['python_scores'])['r_times']),
                json.loads(json.loads(response)['python_times']))

In [ ]:
import timeit

time_test_results = list()
time_test_data_sizes = (1e1, 1e1, 1e3, 1e3, 1e5, 1e5, 3e5, 3e5)
time_test_data_sizes = (1e1, 1e1, 1e3, 1e3, 1e5, 1e5)

def test_service(data_size, scoring_service):
    start_time = timeit.default_timer()

    raw_data = 2 * np.random.random_sample((data_size, 2)) - 1
    aml_jsoned_data =  json.dumps({'data': json.dumps(raw_data.tolist())})
    print('\n data_size: {} rows, jsoned data is {} chars long'.format(data_size, len(aml_jsoned_data)))
    
    start_service_time = timeit.default_timer()
    response = scoring_service.run(input_data = aml_jsoned_data)
    return_service_time = timeit.default_timer()
    
    print( pd.DataFrame.from_records(json.loads(json.loads(response)['python_scores'])['r_times']) )
    
    for k, v in json.loads(json.loads(response)['python_times']).items():
        print(v, k)
    
    end_time = timeit.default_timer()
    for crt_key, crt_value in \
    {'e2e_time':'{} ms'.format(round((end_time-start_time)*1000, 2)),
          'service_time':'{} ms'.format(round((return_service_time-start_service_time)*1000, 2)),
          'data_generation_time':'{} ms'.format(round((start_service_time-start_time)*1000, 2)),
          'response_print_time':'{} ms'.format(round((end_time-return_service_time)*1000, 2))}.items():
        print(crt_key, ': ',crt_value)
    
    rpy_times_report(pd.DataFrame.from_records(json.loads(json.loads(response)['python_scores'])['r_times']),
                json.loads(json.loads(response)['python_times']))
    
# aci proper testing moved near aks testing, below

In [ ]:
# aci_service.delete()

### Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()


In [ ]:
for crt_compute_target in ComputeTarget.list(workspace = ws):
    print(crt_compute_target.name)
#     print(crt_compute_target.cluster_resource_id)
    print(crt_compute_target.type)
    print(crt_compute_target.description)
    print(crt_compute_target.get_status())

In [ ]:
# !az aks get-credentials -n r-aks-clst03f0d01421a14ff1 -g $project_new_rsg -a -f r-aks-clst03.txt

In [ ]:
aks_cluster_name = 'ro16n-aks-001'

# Create the AKS cluster. Existing clusters will be reused
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

In [ ]:
%%time
aks_cluster_name
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

In [ ]:
resource_id = aks_target.cluster_resource_id

### Optional step: Attach existing AKS cluster
If you have existing AKS cluster in your Azure subscription, you can attach it to the Workspace.

In [ ]:
%%time
# Use the default configuration (can also provide parameters to customize)

attach_cluster = False
if (attach_cluster):
    # attach existing  cluster
    
    attach_config = AksCompute.attach_configuration(resource_id=resource_id)
    aks_target = ComputeTarget.attach(workspace=ws, name=aks_cluster_name, attach_configuration=attach_config)
    # Wait for the operation to complete
    aks_target.wait_for_completion(True)

### Deploy web service to AKS

In [ ]:
#Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

In [ ]:
%%time

from azureml.exceptions import WebserviceException
try:
    aks_service = Webservice.deploy_from_image(workspace = ws, 
                                               name = aks_service_name,
                                               image = image_to_deploy,
                                               deployment_config = aks_config,
                                               deployment_target = aks_target)
    aks_service.wait_for_deployment(show_output = True)
    print(aks_service.state)
except WebserviceException:
    print('WebserviceException: There is already a service with name {} found in workspace {}. Will use it, and not create another one!'\
          .format(aks_service_name, ws.name))

In [ ]:
# list all web services in the workspace
for s in ws.webservices:
    print(s)

aks_service = Webservice(workspace = ws, name = aks_service_name)

In [ ]:
# aks_service.get_logs()

In [ ]:
time_test_data_sizes = (1e1, 1e1, 1e3, 1e3, 1e5, 1e5)
for time_test_data_size in time_test_data_sizes:
    test_service(int(time_test_data_size), aci_service)

In [ ]:
for time_test_data_size in time_test_data_sizes:
    test_service(int(time_test_data_size), aks_service)

In [ ]:
# # # Clean-up
# aci_service.delete()
# aks_service.delete()

In [ ]:
!jupyter nbconvert --to html  020_RegularR_RealTime_deploy_ACI_AKS.ipynb

In [ ]:
print('Finished running 020_RegularR_RealTime_deploy_ACI_AKS.ipynb!')